In [1]:
import numpy as np

In [2]:
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
# Select GPU Number
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
# check if cuda available
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(315)
if device == "cuda":
    torch.cuda.manual_seed_all(912)

device

'cuda'

In [5]:
df = np.loadtxt("./ml-1m/ratings.dat", delimiter ='::')
df = df[:, :3]

In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=1981)

In [7]:
print(df_train[:,0].max(), df_train[:,1].max())

6040.0 3952.0


In [8]:
train_x = df_train[:, :2]
train_y = df_train[:, 2]
train_x[:,1] = train_x[:,1] + train_x[:,0].max()

In [9]:
class MovielensDataset(Dataset):
    """
    torch.utils.data.Dataset 상속
    """
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [10]:
train_dataset = MovielensDataset(X = torch.LongTensor(train_x),
                                y = torch.FloatTensor(train_y))

In [11]:
class FactorizationMachine(nn.Module):
    
    def __init__(self, field_dims, latent_dims):
        super(FactorizationMachine, self).__init__()
        
        self.w_0 = nn.Parameter(torch.zeros((1, )), requires_grad=True)
        self.w_i = nn.Embedding(num_embeddings = field_dims, embedding_dim = 1)
        self.V = nn.Embedding(num_embeddings = field_dims, embedding_dim = latent_dims)
        nn.init.xavier_uniform_(self.V.weight.data)
        
        
    def forward(self, x):
        
        temp_1 = torch.sum(self.w_i(x), dim=1) + self.w_0
        
        square_of_sum = torch.sum(self.V(x), dim = 1) ** 2
        sum_of_square = torch.sum(self.V(x) ** 2, dim = 1)
        ix = square_of_sum - sum_of_square
        temp_2 = torch.sum(ix, dim=1, keepdim=True)
        
        result = temp_1 + 0.5 * temp_2
        
        return result
    
    def init_weight(self):
        pass

In [12]:
model = FactorizationMachine(field_dims = int(train_x[:,1].max() + 1), latent_dims = 40).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()
batch_size = 64
n_epochs = 100

In [13]:
for epoch_id in range(n_epochs):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):

        X, y = batch[0], batch[1]
        X, y = X.cuda(), y.cuda()

        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_function(y_pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss
        
    model.eval()
    print ('Epoch {} of {}, training Loss: {:.4f}'.format(epoch_id + 1, n_epochs, total_loss))

/home/namkyeong/anaconda3/envs/mf_v1/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/namkyeong/anaconda3/envs/mf_v1/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([39])) that is different to the input size (torch.Size([39, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 of 100, training Loss: 32255.1719


KeyboardInterrupt: 